In [2]:
import sys
STATA_SYSDIR =  "/Applications/StataNow"
sys.path.append(f'{STATA_SYSDIR}/utilities')

from pyutils.econometrics import stata

In [ ]:

from pystata import stata
import pandas as pd
def ivreg(estimator:str, depvar:str, exog:list[str], endog:str, instruments:list[str], data:pd.DataFrame):
    stata.pdataframe_to_data(data, force=True)
    stata.run(f'ivregress {estimator} {depvar} {" ".join(exog)} ({endog} = {" ".join(instruments)})')
    return  stata.get_ereturn()

In [8]:
data = pd.DataFrame({
    'y': [1,2,3,4,5,6,7,8,9,10],
    'x1': [2,3,4,5,6,7,8,9,10,11],
    'x2': [5,3,6,2,7,4,8,1,9,0],
    'z': [10,9,8,7,6,5,4,3,2,1],
    'fe': [1,1,1,1,1,2,2,2,2,2]
})  

# result = ivreg('2sls', 'y', ['x1', 'x2'], 'x1', ['z'], data)
# print(result)

In [12]:
stata.pdataframe_to_data(data, force=True)
stata.run('xtset fe')
stata.run(f'xtreg y x1 x2, fe')
stata.get_ereturn()

{'e(r2_c)': 1.0,
 'e(F_f)': 8.98846567431158e+307,
 'e(p_f)': 8.98846567431158e+307,
 'e(Tbar)': 5.0,
 'e(Tcon)': 1.0,
 'e(g_min)': 5.0,
 'e(g_avg)': 5.0,
 'e(g_max)': 5.0,
 'e(N_g)': 2.0,
 'e(k_absorb)': 2.0,
 'e(rc)': 0.0,
 'e(ui)': 1.444485061989073e-15,
 'e(sigma_u)': 1.444485061989073e-15,
 'e(corr)': -0.8703882797784892,
 'e(r2_o)': 1.0,
 'e(r2_b)': 1.0,
 'e(sigma_e)': 0.0,
 'e(sigma)': 1.444485061989073e-15,
 'e(rho)': 1.0,
 'e(rank)': 0.0,
 'e(p)': 8.98846567431158e+307,
 'e(r2_w)': 1.0,
 'e(df_m)': 3.0,
 'e(df_b)': 2.0,
 'e(ll_0)': -17.655121234846455,
 'e(tss)': 82.5,
 'e(df_a)': 1.0,
 'e(r2_a)': 1.0,
 'e(rss)': 0.0,
 'e(mss)': 20.0,
 'e(rmse)': 0.0,
 'e(r2)': 1.0,
 'e(F)': 8.98846567431158e+307,
 'e(df_r)': 6.0,
 'e(sum_w)': 10.0,
 'e(N)': 10.0,
 'e(cmdline)': 'xtreg y x1 x2, fe',
 'e(estat_cmd)': 'xtreg_estat',
 'e(_r_z_abs__CL)': '|t|',
 'e(_r_z__CL)': 't',
 'e(cmd)': 'xtreg',
 'e(marginsprop)': 'nolinearize',
 'e(marginsnotok)': 'E U UE SCore STDP XBU XBD D',
 'e(predict)